In [ ]:
import os
import boto3
from progressbar import ProgressBar
from random import randint
from pathlib import Path
from common.pix4d_libs import get_jwt, create_project, project_s3_creds, register_images, start_processing, get_project

In [ ]:
PIX4D_CLIENT_ID = os.environ['PIX4D_CLIENT_ID']
PIX4D_CLIENT_SECRET = os.environ['PIX4D_CLIENT_SECRET']
assert PIX4D_CLIENT_ID
assert PIX4D_CLIENT_SECRET

In [ ]:
my_jwt = get_jwt(PIX4D_CLIENT_ID, PIX4D_CLIENT_SECRET)

In [ ]:
print(my_jwt)

In [ ]:
project_id = create_project(f'demo {randint(0, 1000)}', my_jwt)['id']

In [ ]:
s3_creds = project_s3_creds(project_id, my_jwt)

In [ ]:
s3_creds

In [ ]:
s3_client = boto3.client('s3', 
                         aws_access_key_id=s3_creds['access_key'], 
                         aws_secret_access_key=s3_creds['secret_key'],
                         aws_session_token=s3_creds['session_token'])

In [ ]:
keys = []
images = list(Path('images/').glob("*.JPG"))
with ProgressBar(max_value=len(images)) as pbar:
    for i, image in enumerate(images):
        k = str(Path(s3_creds['key']) / Path(image).name)
        s3_client.put_object(
            Bucket=s3_creds['bucket'], 
            Key=k, 
            Body=Path(image).read_bytes(),
            ACL="bucket-owner-full-control"
        )
        keys.append(k)
        pbar.update(i)

In [ ]:
keys

In [ ]:
register_images(project_id, my_jwt, keys)

In [ ]:
start_processing(project_id, my_jwt)

In [ ]:
print(project_id)

In [ ]:
print(get_project(project_id, my_jwt)['public_status'])